In [0]:
import requests

def extract_data(date, base="BRL"):

    url = f"https://api.apilayer.com/exchangerates_data/{date}&base={base}"

    headers= {
    "apikey": "26fIuqF2C5C9BA52xsBxEF8YaAiXnddd"
    }

    parameters = {
        "base":base,
        "symbols": "USD,GBP,EUR"
        }
    

    response = requests.request("GET", url, headers=headers, params=parameters)

    if response.status_code != 200:
        raise Exception(f'Request failed with status {response.status_code}')
    return response.json()


In [0]:
extract_data("2025-07-20")

In [0]:
year, month, day = extract_data("2025-07-20")['date'].split('-')
print(year, month, day)

In [0]:
display(dbutils.fs.ls("dbfs:/"))

In [0]:
path = f"dbfs:/databricks-results/bronze/{year}/{month}/{day}"
print(path)

In [0]:
def data_to_df (data_json): 
    data_tuple = [(moeda, float (taxa)) for moeda, taxa in data_json["rates"].items()]
    
    return data_tuple

In [0]:
response = data_to_df(extract_data("2025-07-20"))
response

In [0]:
df_conversions = spark.createDataFrame(response, schema=["moeda", "taxa"])
display(df_conversions)

In [0]:
from pyspark.sql.functions import lit

df_conversions = df_conversions.withColumn("date", lit(f"{year}-{month}-{day}"))

In [0]:
display(df_conversions)

In [0]:
df_conversions.write.format("parquet").mode("overwrite").save(path)

In [0]:
display(dbutils.fs.ls(path))